In [2]:
import requests
import json
import pandas as pd
import feedparser as fp
import newspaper as np

def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories):
    # Liste des nouveaux articles
    new_articles = []

    # Parcourir les flux RSS
    for source_url in flux_rss_a_lire:
        # Déterminer la catégorie du flux
        category = categories[source_url]

        data = fp.parse(source_url)
        
        # Parcourir les articles du flux RSS
        for item in data.entries:
            # Clés renommées pour correspondre à la base de données existante
            article = {
                "title": item.title,
                "date": item.published,  # Renommé de 'published' à 'date'
                "url": item.link,  # Renommé de 'link' à 'url'
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": category,  # Ajouté la catégorie
            }
            
            # Vérifier si l'article existe déjà dans la base de données
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                
                # Mettre à jour les informations de l'article
                article["content"] += "\n\n" + article_obj.text  # Ajouter le contenu téléchargé
                article["author"] = article_obj.authors  # Ajouté la liste des auteurs
                
                # Ajouter l'article à la liste des nouveaux articles
                new_articles.append((article["url"], article))

    # Mettre à jour la base de données existante avec les nouveaux articles
    for url, article in new_articles:
        database[url] = article

    return database


# Lien vers le fichier tvinfo-sources.json
tvinfo_sources_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json"

# Charger la liste des flux RSS depuis le lien tvinfo-sources.json
response_sources = requests.get(tvinfo_sources_url)
sources = json.loads(response_sources.text)

# Liste des flux RSS à parcourir
flux_rss_a_lire = list(sources.values())

# Usage de la fonction avec les catégories
categories = {
    "https://www.francetvinfo.fr/france.rss": "france",
    "https://www.francetvinfo.fr/monde/europe.rss": "europe",  # Corrigé ici
    "https://www.francetvinfo.fr/economie/entreprises.rss": "economie"
}


# Charger la base de données existante depuis le lien francetvinfo.json
francetvinfo_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json"
response_database = requests.get(francetvinfo_url)
database = json.loads(response_database.text)

# Mettre à jour la base de données avec les nouveaux articles
database_mise_a_jour = mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories)

# Transformer la base de données mise à jour en DataFrame
df_database_mise_a_jour = pd.DataFrame(database_mise_a_jour.values())

# Ajouter les URLs comme colonne dans le DataFrame
df_database_mise_a_jour['url'] = list(database_mise_a_jour.keys())

# Afficher les premières lignes pour vérifier
df_database_mise_a_jour





,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2152,Comment féminiser les métiers de terrain dans ...,"Sun, 11 Feb 2024 08:47:43 +0100",[Sarah Lemoine],economie,La part des femmes a légèrement progressé dans...,https://www.francetvinfo.fr/pictures/-GnreVvJ2...,NaN,https://www.francetvinfo.fr/replay-radio/c-est...
2153,Entreprises : la crise de l’immobilier pèse su...,"Fri, 09 Feb 2024 09:14:50 +0100",[Emmanuel Cugny],economie,"Les Français ont moins acheté de lave-linges, ...",https://www.francetvinfo.fr/pictures/AnPZT-kFs...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2154,"Semaine de quatre jours : ""Il va falloir ident...","Thu, 08 Feb 2024 20:19:53 +0100","[Emmanuel Cugny, Marie Bouny]",economie,"Marie Bouny, docteure en droit et associée au ...",,NaN,https://www.francetvinfo.fr/replay-radio/l-int...
2155,"Nature des travaux, montant des projets : quel...","Thu, 08 Feb 2024 18:01:04 +0100",[Lauriane],economie,Pour encourager les entreprises privées à s'ad...,https://www.francetvinfo.fr/pictures/NIA-U7YVA...,NaN,https://www.francetvinfo.fr/monde/environnemen...


# Mettre à jour la base de données de Git

In [3]:

# Fonction pour récupérer la base de données actuelle depuis GitHub
def telecharger_base_donnees_github(url):
    response = requests.get(url)
    data = response.text.splitlines()  # Séparer chaque ligne du fichier JSON
    df = pd.DataFrame([pd.read_json(line, typ='series') for line in data])
    return df

# Fonction pour concaténer l'ancienne base de données avec les nouveaux articles
def concatener_bases_donnees(df_ancienne, df_nouveaux_articles):
    # Filtrer les nouveaux articles pour ne garder que ceux qui n'existent pas déjà dans l'ancienne base de données
    nouveaux_articles_uniques = df_nouveaux_articles[~df_nouveaux_articles['url'].isin(df_ancienne['url'])]
    # Concaténer les deux DataFrames
    df_mise_a_jour = pd.concat([df_ancienne, nouveaux_articles_uniques], ignore_index=True)
    return df_mise_a_jour

# URL de la base de données sur GitHub
url_github = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'

# Charger la base de données actuelle
df_ancienne = telecharger_base_donnees_github(url_github)

# Concaténer l'ancienne base de données avec les nouveaux articles
df_mise_a_jour = concatener_bases_donnees(df_ancienne, df_database_mise_a_jour)

# Sauvegarder la base de données mise à jour localement
df_mise_a_jour.to_json("Data/francetvinfo_maj.json", orient="records", lines=True, force_ascii=False)
df_mise_a_jour.to_csv("Data/francetvinfo_maj.csv", index=False)


In [4]:
df_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2231,"""Doppelgänger"", articles détournés et fake new...","Wed, 21 Feb 2024 11:21:50 +0100",[France],europe,Le conflit lancé il y a deux ans se double d'u...,https://www.francetvinfo.fr/pictures/qW-MUyAxU...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2232,"""C'est tout aussi incroyable et excitant pour ...","Wed, 21 Feb 2024 09:54:42 +0100",[Angélique Bouin],europe,Des ouvriers ont eu la surprise de découvrir d...,https://www.francetvinfo.fr/pictures/QtZP3dISr...,NaN,https://www.francetvinfo.fr/monde/europe/belgi...
2233,"REPORTAGE. ""Nous sommes la cible numéro un"" : ...","Wed, 21 Feb 2024 08:44:44 +0100","[Vanessa Descouraux, Gilles Gallinaro, Inna Sh...",europe,"Dans la maternité de Pokrovsk, la dernière enc...",https://www.francetvinfo.fr/pictures/iFWeTAyEz...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2234,Guerre en Ukraine : le G7 discutera samedi de ...,"Wed, 21 Feb 2024 00:09:50 +0100",[],europe,Les membres de groupe avaient décidé en juille...,https://www.francetvinfo.fr/pictures/bjVBo9EhH...,NaN,https://www.francetvinfo.fr/monde/europe/manif...


In [5]:
import pandas as pd
# URL du fichier mise à jour chaque jour JSON sur GitHub
url = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'
# Charger les données dans un DataFrame
francetvinfo_mise_a_jour = pd.read_json(url, lines=True)
# Afficher les premières lignes du DataFrame
francetvinfo_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2213,"Marc Aublet, PDG de Provalliance : ""On a passé...","Mon, 19 Feb 2024 21:34:41 +0100","[Camille Revel, Marc Aublet, Pdg De Provalliance]",economie,"Le groupe Provalliance, qui comprend les marqu...",,None,https://www.francetvinfo.fr/replay-radio/l-int...
2214,VIDEO. Guerre en Ukraine : comment la Younarmi...,"Tue, 20 Feb 2024 20:57:12 +0100",[],europe,L’analyse de centaines de vidéos sur les résea...,,None,https://www.francetvinfo.fr/monde/europe/manif...
2215,Europol se félicite de l'opération contre les ...,"Tue, 20 Feb 2024 18:31:25 +0100",[Richard Place],europe,"Jean-Philippe Lecouffe, directeur exécutif adj...",https://www.francetvinfo.fr/pictures/PfhtuMkGs...,None,https://www.francetvinfo.fr/internet/securite-...
2216,Guerre en Ukraine : pourquoi la situation sur ...,"Tue, 20 Feb 2024 17:45:02 +0100","[Valentine Pasquesoone, Le Centre D'Analyse In...",europe,"Le président ukrainien s'est inquiété lundi, d...",https://www.francetvinfo.fr/pictures/le9S4oMDC...,None,https://www.francetvinfo.fr/monde/europe/manif...
